In [1]:
import re
import pandas as pd
import numpy as np
import os, shutil
import inspect
import datetime as dt
from sys import platform
from glob import glob
from tqdm import tqdm
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

import seronetDataclass as seroClass
import seronetFunctions as seroFxn

In [10]:
cloud = '/Users/liualg/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Curation channel/ImmPort Uploads/'
# PMID_LIST = pd.read_excel(os.path.join(cloud,'Immport-DR46 (November).xlsx'))['PMID (link to Box)'][1:]
PMID_LIST = [
33767426,
34133941,
34688034,
35764089,
35605428,
34594341,
34861167,
32678685,
32941787,
35857529,
33822097,
36480499,
33798476,
34951746,
37264688,
34862384,
34942634,
36597886,
35102312,
36208629,
37863065,
36192374,
37695910
]
DATA_RELEASE='51'
OUT_DIR = f'./InternalQs/DR{DATA_RELEASE}'
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

In [28]:
# finding correct Box Base
if platform == "darwin":
    box_base = "/Users/liualg/Library/CloudStorage/Box-Box/SeroNet Curation/SeroNet Public Data"
else: 
    print("User has windows")
    box_base = os.path.join("Users",os.getlogin(), "Box")


In [29]:
# create JSONs
def create_json(PMID, OUT_DIR):
    
#     print(box_base)
    BASE_DIR = seroFxn.get_box_dir(box_base, PMID)
#     print(BASE_DIR)
    df_path = glob(os.path.join(BASE_DIR,'templated_data',f'PMID{PMID}_'))
    df_path.sort()
    df_path = df_path[0]
    
    file_type = 'json'
    output_file = os.path.join(OUT_DIR, f'PMID{PMID}_JSON.{file_type}')
    df = pd.read_excel(df_path, sheet_name = 0, header=None)
    df.index += 1
    template = {}
    pt.parse_registry_template(df, template)

    f = open(output_file, "w")
    print(json.dumps(template, indent=4), file = f)
    f.close()

In [34]:
def get_all_files(pmid):
    if isinstance(pmid, int):
        pmid = str(pmid)
    else:
        pass
    #File Paths
    BASE_DIR = seroFxn.get_box_dir(box_base, pmid)
    OUT_DIR = glob(os.path.join(BASE_DIR,f'*{DATA_RELEASE}*'))
    OUT_DIR.sort()
#     print(OUT_DIR)
    
    for file in glob(os.path.join(OUT_DIR[-1],"*.txt")):
        if 'protocol.txt' in file:
            protocols.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'experiments' in file:
            experiments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'reagent' in file:
            reagents.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'panel' in file:
            assessments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'subject_human' in file:
            subject_humans.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'subject_organism' in file:
            subject_organisms.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'experiment_samples' in file:
            experiment_samples.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'treatment' in file:
            treatments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'basic' in file:
            basic_stdy_templates.append(file)
        else:
#             print(OUT_DIR[-1])
            pass

#     return {'basic_stdy_templates':basic_stdy_templates, 'protocols': protocols, 'experiments': experiments,
#             'reagents': reagents,'assessments': assessments,'subject_humans': subject_humans,
#             'subject_organisms': subject_organisms, 'experiment_samples': experiment_samples,
#             'treatments': treatments
#            }

In [35]:
def get_sections(ws, class_names):
    temp = []
    for i in range(1,ws.max_row):
        if ws["A"][i].value in class_names:
            temp.append(i+1)
    return temp

def replace_header(dataframe):
    dataframe.columns = dataframe.iloc[0]
    return dataframe[1:]

# def clean_df(dataframe, add_PMID = False):
    
#     dataframe = seroFxn.edit_df(dataframe)
#     dataframe = dataframe.dropna(how='all',axis=0).dropna(how='all',axis=1)
#     if add_PMID:
#         dataframe['PMID'] = [PMID]*len(dataframe)
#     else:
#         pass
    
#     reut
# #             df['PMID'] = [PMID]*len(df) #LIU
# #             CAT.append(df)



#     dataframe = seroFxn.edit_df(dataframe)
#     dataframe = dataframe.dropna(how='all',axis=0).dropna(how='all',axis=1)
#     if add_PMID:
#         dataframe['PMID'] = [PMID]*len(dataframe)
    
#     return replace_header(dataframe.reset_index())

def get_basic_info(ws, PMID):
    # Class names
    class_names = ['study', 'study_personnel', 'study_categorization', 'study_2_condition_or_disease',
                       'arm_or_cohort', 'study_personnel', 'planned_visit',
                       'inclusion_exclusion', 'study_2_protocol', 'study_file',
                       'study_link','study_pubmed'
                  ]



    VARS_TO_CLEAN = ['', 'N/A', 'n/a', 'na', np.nan, None]
    clean_other = VARS_TO_CLEAN + ['Other']

    ws.delete_cols(1)  
    sp = get_sections(ws, class_names)
    sp.append(200)
    
    for section_number in range(len(sp)-1):
        temp_wb = openpyxl.Workbook()
        temp_ws = temp_wb.active

        #making a temp workbook to store each section. This will be turned into df
        for i in ws.iter_rows(values_only = True,
                                    min_row = sp[section_number]+1,
                                    max_row = sp[section_number+1]-1):
            temp_ws.append(i)
            
#         max_row = temp_ws.max_row
#         max_col = temp_ws.max_column
        seroFxn.remove_excess(temp_ws)
        
        df = pd.DataFrame(temp_ws.values)
        sub_section = ws.cell(row=sp[section_number], column = 1).value.strip()
#         print(sub_section)
        if sub_section == 'study':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all')
            STUDY.append(df)
            
        elif sub_section == 'study_categorization':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            df['PMID'] = [PMID]*len(df) #LIU
            CAT.append(df)
            
        elif sub_section == 'study_2_condition_or_disease':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            df['PMID'] = [PMID]*len(df)
            COD.append(df)
            
        elif sub_section == 'inclusion_exclusion':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            IE.append(replace_header(df.reset_index()))
            
        elif sub_section == 'arm_or_cohort':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            AOC.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_personnel':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            PERSONNEL.append(replace_header(df.reset_index()))
            
        elif sub_section == 'planned_visit':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            PV.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_file':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            FILE.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_link':
            df = seroFxn.edit_df(df).T
            if df.empty:
                pass
            else:
                df = df.dropna(how='all').dropna(how='all',axis=1)
                df = replace_header(df)
                df['PMID'] = [PMID]*len(df)
                LINK.append(df.reset_index())
            
        else:
            pass
        
#     return {
#      'STUDY':STUDY,'CAT':CAT,'COD':COD,'IE':IE,'AOC':AOC,'PERSONNEL' :PERSONNEL,'PV' :PV,'FILE' :FILE, 'LINK' :LINK
#     }


In [36]:
# create_json(OUT_DIR, "35180044")

In [39]:
#templates excluding study
basic_stdy_templates = []
protocols = []
experiments = []
reagents = []
assessments = []
subject_humans = []
subject_organisms = []
experiment_samples = []
treatments = []

for PMID in tqdm(PMID_LIST):
    PMID = str(PMID)
    try:
        get_all_files(PMID)
    except:
        pass

# basic_stdy_template_report = []
protocols_report = pd.concat(protocols)
# reagent_report = pd.concat(reagents)
# assessment_report = pd.concat(assessments)
# subject_human_report = pd.concat(subject_humans)
# subject_organism_report = pd.concat(subject_organisms)
experiment_report = pd.concat(experiments)
# treatment_report = pd.concat(treatments)    

100%|████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  4.52it/s]


In [40]:
#study split
STUDY  = []
CAT = []
COD = []
IE = []
AOC = []
PERSONNEL = []
PV = []
FILE = []
LINK = []

for basic_path in tqdm(basic_stdy_templates):
    basic_wb = openpyxl.Workbook()
    basic_ws = basic_wb.active

    for r in dataframe_to_rows(pd.read_csv(basic_path, sep='\t',header=None)):
        basic_ws.append(r)

    get_basic_info(basic_ws, re.findall(r'\d{8}',basic_path)[0])

100%|████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 13.93it/s]


In [43]:
# basic_stdy_template_report = []

protocols_report = pd.concat(protocols).reset_index(drop=True)
# reagent_report = pd.concat(reagents).reset_index(drop=True)
# assessment_report = pd.concat(assessments).reset_index(drop=True)
# subject_human_report = pd.concat(subject_humans).reset_index(drop=True)
# subject_organism_report = pd.concat(subject_organisms).reset_index(drop=True)
experiment_report = pd.concat(experiments).reset_index(drop=True)
# treatment_report = pd.concat(treatments).reset_index(drop=True)

STUDY_report = pd.concat(STUDY).dropna(axis=1,how='all').reset_index(drop=True)
COD_report = pd.concat(COD).reset_index(drop=True)
CAT_report = pd.concat(CAT).reset_index(drop=True)
IE_report = pd.concat(IE).reset_index(drop=True)
AOC_report = pd.concat(AOC).reset_index(drop=True)
PERSONNEL_report = pd.concat(PERSONNEL).reset_index(drop=True)
PV_report = pd.concat(PV).reset_index(drop=True)
FILE_report = pd.concat(FILE).reset_index(drop=True)
LINK_report = pd.concat(LINK).reset_index(drop=True)
LINK_report.columns = ['Name','Value','PMID']

In [44]:
# subject_human_report

In [45]:
protocols_report.to_csv(f'./{OUT_DIR}/protocols_report.csv')
# reagent_report.to_csv(f'./{OUT_DIR}/reagent_report.csv')
# assessment_report.to_csv(f'./{OUT_DIR}/assessment_report.csv')
# subject_human_report.to_csv(f'./{OUT_DIR}/subject_human_report.csv')
# subject_organism_report.to_csv(f'./{OUT_DIR}/subject_organism_report.csv')
experiment_report.to_csv(f'./{OUT_DIR}/experiment_report.csv')
# treatment_report.to_csv(f'./{OUT_DIR}/treatment_report.csv')
STUDY_report.to_csv(f'./{OUT_DIR}/STUDY_report.csv')
COD_report.to_csv(f'./{OUT_DIR}/COD_report.csv')
CAT_report.to_csv(f'./{OUT_DIR}/CAT_report.csv')
IE_report.to_csv(f'./{OUT_DIR}/IE_report.csv')
AOC_report.to_csv(f'./{OUT_DIR}/AOC_report.csv')
PERSONNEL_report.to_csv(f'./{OUT_DIR}/PERSONNEL_report.csv')
PV_report.to_csv(f'./{OUT_DIR}/PV_report.csv')
FILE_report.to_csv(f'./{OUT_DIR}/FILE_report.csv')
LINK_report.to_csv(f'./{OUT_DIR}/LINK_report.csv')

In [13]:
# from collections import Counter 
# Counter(experiment_report['Name'])